In [5]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml.html as lh



In [6]:
URL = "https://projects.fivethirtyeight.com/2021-mlb-predictions/games/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [7]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

[len(T) for T in tr_elements[:11]]


[9, 9, 8, 9, 8, 9, 8, 9, 8, 9, 8]

In [8]:
len(tr_elements)

402

In [9]:
upcoming_list_done = False
future_list_done = False

rows = [row.text_content() for row in tr_elements]

header_str = "DateTeamStarting pitcherTeam ratingStarting pitcher adj.Travel, rest & home field adj.Pregame team ratingWin prob.Chance of winningScore"

rows.index(header_str)

indexes = [i for i,row in enumerate(rows) if row == header_str]

## This gives us the two header strings, telling us that row 0 is a header
## and 1-200 are the body rows for the first table
## 201 is a header, and 202-end are the body rows for the 2nd table
indexes

[0, 201]

#### Now that we have the structure of the two tables, we can start segmenting them and getting actual prediction data

In [12]:
upcoming_games_header = rows[0]
upcoming_games_rows = rows[1:201]

previous_games_header = rows[201]
previous_games_rows = rows[201:]

In [62]:
game_predictions = []

for i in range(0,len(upcoming_games_rows),2):
    
    row_w_game_info = upcoming_games_rows[i]
    row_w_team_only = upcoming_games_rows[i + 1]
        
    date = re.search("[0-9]{1,2}\/[0-9]{1,2}", row_w_game_info).group()
    away_team = re.search("[A-Z]{3}", row_w_game_info).group()
    home_team = re.search("[A-Z]{3}", row_w_team_only).group()
    
    ## So far, there are no instances of single-digit percentage predictions from 538, but I'll  
    away_pct_538_str = re.search("[0-9]{5,6}%", row_w_game_info).group()
    if len(away_pct_538_str) == 7:
        away_pct_538 = away_pct_538_str[-3:]
    elif len(away_pct_538_str) == 6:
        away_pct_538 = away_pct_538_str[-2:]  
        
    home_pct_538_str = re.search("[0-9]{5,6}%", row_w_team_only).group()
    if len(home_pct_538_str) == 7:
        home_pct_538 = home_pct_538_str[-3:]
    elif len(home_pct_538_str) == 6:
        home_pct_538 = home_pct_538_str[-2:]

        
    prediction_row = [date,away_team,home_team,away_pct_538,home_pct_538]
    
    game_predictions.append(prediction_row)
    
game_predictions_df = pd.DataFrame(game_predictions, columns=['Date', 'away_team', 'home_team','away_pct_538','home_pct_538'])

game_predictions_df.head()

,Date,away_team,home_team,away_pct_538,home_pct_538
0,8/27,TBR,BAL,69%,31%
1,8/27,ARI,PHI,37%,63%
2,8/27,STL,PIT,56%,44%
3,8/27,BOS,CLE,57%,43%
4,8/27,TOR,DET,56%,44%


In [66]:
game_predictions_df.to_csv("538_scrape_output.csv", index=False)